<a href="https://colab.research.google.com/github/TTTTTTT114/Algo_Trading/blob/master/QuickStockAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Goal

What we need to implement in the first step

5 Basic Quant Strategies:
1. Mean Reversion
2. Momentum
3. Valuation
4. Sentiment
5. Seasonality

Source of Lectures and Algos:

https://www.quantopian.com/lectures

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import datetime
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


# 2. Data Source

Yahoo finance

In [0]:
% pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=5f64c9009f01a8dddedd10bca5ccbb49e66023eaa5ad008f237520eb6db7d9c6
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [0]:
% pip install quandl
import quandl

 ** **Quandl verification key** **


 ** bv8Qp32xTuis-Tx7qjmY **

In [0]:
# Setting API
quandl.ApiConfig.api_key = "bv8Qp32xTuis-Tx7qjmY"

**Yahoo Finance Data**

In [0]:
import yfinance as yf

In [0]:
tsla = yf.download('TSLA', start='2020-01-01', end='2020-04-27')

[*********************100%***********************]  1 of 1 completed


In [0]:
tsla.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,424.500000,430.700012,421.709991,430.260010,430.260010,9532100
2020-01-03,440.500000,454.000000,436.920013,443.010010,443.010010,17778500
2020-01-06,440.470001,451.559998,440.000000,451.540009,451.540009,10133000
2020-01-07,461.399994,471.630005,453.359985,469.059998,469.059998,17882100
2020-01-08,473.700012,498.489990,468.230011,492.140015,492.140015,31144300


# Fundamentals

### Hypothesis Testing

The four most common distributions for test statistics are as follows:

* The  t -distribution ( t -test)
* The standard normal distribution ( z -test)
* The chi-square ( χ2 ) distribution ( χ2 -test)
* The  F -distribution ( F -test)



eg. for a z-test:

Assume normal distribution(Be careful when applying the Central Limit Theorem, however, as many datasets in finance are fundamentally non-normal and it is not safe to apply the theorem casually or without attention to subtlety.) 

$$
Z = \frac{(x - \mu)}{\sigma / \sqrt{n}}
$$

x sample mean

mu target value

sigma sample variance

n sample size

**The following are the main steps in hypothesis testing:**

1. State the hypothesis and the alternative to the hypothesis
2. Identify the appropriate test statistic and its distribution. Ensure that any assumptions about the data are met (stationarity, normality, etc.)
3. Specify the significance level,  α 
4. From  α  and the distribution compute the 'critical value'.
5. Collect the data and calculate the test statistic
6. Compare test statistic with critical value and decide whether to accept or reject the hypothesis.

- null hypothesis H0

- alternative hypothesis H1

- confidence interval 95% (conventional sig_level: 0.1, 0.05, 0.01)

The most common type of hypothesis test is the two-tailed, "not equal to", hypothesis test, because it presents a neutral view. **The one-tailed hypothesis tests are less neutral than the "not equal to" test**, reflecting the thoughts of the tester. One-tailed tests are often used to test "hoped for" results or results that the testers have a **prior idea** about.

In [0]:
#                                 True Situation

# Decision                        H0 True                 H0 False

# Do not reject H0                Correnct Decision       Type 2 Error

# Reject H0 (Accept HA)           Type 1 Error            Correct Decision

![alt text](https://sixsigmastudyguide.com/wp-content/uploads/2014/04/z-table.jpg)

##Performance Measurement - KPIs

Daily Return 

In [0]:
'''
Close_next_day / Close_current_day - 1

'''

In [0]:
tsla['Daily Returns'] = tsla['Adj Close'].pct_change()

In [0]:
tsla.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Returns
Date,,,,,,,
2020-01-02,424.500000,430.700012,421.709991,430.260010,430.260010,9532100,NaN
2020-01-03,440.500000,454.000000,436.920013,443.010010,443.010010,17778500,0.029633
2020-01-06,440.470001,451.559998,440.000000,451.540009,451.540009,10133000,0.019255
2020-01-07,461.399994,471.630005,453.359985,469.059998,469.059998,17882100,0.038801
2020-01-08,473.700012,498.489990,468.230011,492.140015,492.140015,31144300,0.049205


CAGR (Conpound Annual Growth Rate)

In [0]:
'''

CAGR = (V_final/V_begin)^(1/t) - 1

V_final is the final value of the asset
V_begin is the beginning value of the asset
t is the time in years

'''

In [0]:
# def a function that calculate the CAGR
# This is a single value of CAGR
def CAGR(start, end, period):
    V_begin = start
    V_final = end
    t = period
    return (V_final/V_begin)**(1/t) - 1

In [0]:
# example 
CAGR(10000, 19000, 3)

0.23856232963017088

Volatility Measure